In [1]:
# from IPython.display import clear_output
# !pip install transformers datasets torch evaluate
# clear_output()

In [2]:
# importing libraries 
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

In [3]:
pd.read_csv(r'data\TextClassification\IMDB\train.csv')

,label,text
0,1,Four teenage girls in a suburb of Los Angeles ...
1,1,I was in a bad frame of mind when I first saw ...
2,0,"There is a word for this sort of film, and tha..."
3,0,This was a blind buy used DVD. It totally kill...
4,0,As a youth pastor I heard good things about th...
...,...,...
24995,0,"If you didn't know better, you would believe t..."
24996,0,Franco Rossi's 1985 six-hour Italian mini-seri...
24997,1,An excellent example of the spectacular Busby ...
24998,1,"In the early 1970s, many of us who had embrace..."


In [4]:
# here in this dataset I found two datasets for training and testing purpose seprately
df_train = pd.read_csv(r'data\TextClassification\IMDB\train.csv')
df_test = pd.read_csv(r'data\TextClassification\IMDB\test.csv')

In [5]:
def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

In [6]:
#Tockenize and removing stopwords,punctuations and other irrelevant characters  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing as mp

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]
    return " ".join(words)


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [7]:
df_train['text'] = df_train['text'].apply(remove_stopwords)

In [8]:
df_test['text'] = df_test['text'].apply(remove_stopwords)

In [9]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [12]:
# Set the format for PyTorch
train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [13]:
training_args = TrainingArguments(
    output_dir=r'logs/OtherModels/bert_ag_results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    overwrite_output_dir=True,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="no"
)

# Load the metrics
import evaluate;
accuracy_metric = evaluate.load('accuracy', trust_remote_code=True)
precision_metric = evaluate.load('precision', trust_remote_code=True)
recall_metric = evaluate.load('recall', trust_remote_code=True)

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Sat May 18 18:36:55 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--precision\4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Sat May 18 18:37:00 2024) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--recall\e40e6e98d18ff3f210f4d0b26fa721bfaa80704b1fdf890fa551cfabf94fc185 (last modified on Sat May 18 18:37:06 2024) since it couldn't be found locally

In [14]:
torch.cuda.empty_cache()
import gc
gc.collect()

62

In [15]:
# Train the model
# trainer.train(resume_from_checkpoint=r"logs/OtherModels/bert_ag_results/last_epoch")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

  0%|          | 0/6250 [00:00<?, ?it/s]

{'loss': 0.3398, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.4}
{'loss': 0.2704, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.8}


  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 0.22095373272895813, 'eval_accuracy': 0.91252, 'eval_precision': 0.9125448424603053, 'eval_recall': 0.91252, 'eval_runtime': 141.6931, 'eval_samples_per_second': 176.438, 'eval_steps_per_second': 8.822, 'epoch': 1.0}
{'loss': 0.2225, 'learning_rate': 1.5200000000000002e-05, 'epoch': 1.2}
{'loss': 0.1723, 'learning_rate': 1.3600000000000002e-05, 'epoch': 1.6}
{'loss': 0.1787, 'learning_rate': 1.2e-05, 'epoch': 2.0}


  0%|          | 0/1250 [00:00<?, ?it/s]

...

{'eval_loss': 0.4005303382873535, 'eval_accuracy': 0.9178, 'eval_precision': 0.9178184214470743, 'eval_recall': 0.9178, 'eval_runtime': 140.384, 'eval_samples_per_second': 178.083, 'eval_steps_per_second': 8.904, 'epoch': 4.0}
{'loss': 0.0357, 'learning_rate': 2.4000000000000003e-06, 'epoch': 4.4}
{'loss': 0.0369, 'learning_rate': 8.000000000000001e-07, 'epoch': 4.8}


  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 0.42088285088539124, 'eval_accuracy': 0.91832, 'eval_precision': 0.9183242836406644, 'eval_recall': 0.91832, 'eval_runtime': 139.157, 'eval_samples_per_second': 179.653, 'eval_steps_per_second': 8.983, 'epoch': 5.0}
{'train_runtime': 2489.744, 'train_samples_per_second': 50.206, 'train_steps_per_second': 2.51, 'train_loss': 0.13363449722290038, 'epoch': 5.0}


  0%|          | 0/1250 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.42088285088539124, 'eval_accuracy': 0.91832, 'eval_precision': 0.9183242836406644, 'eval_recall': 0.91832, 'eval_runtime': 142.579, 'eval_samples_per_second': 175.341, 'eval_steps_per_second': 8.767, 'epoch': 5.0}


In [19]:
trainer.save_model(r"logs/OtherModels/bert_ag_results/last_epoch")

In [ ]:
Evaluation results: {'eval_loss': 0.42088285088539124, 'eval_accuracy': 0.91832, 'eval_precision': 0.9183242836406644, 'eval_recall': 0.91832, 'eval_runtime': 142.579, 'eval_samples_per_second': 175.341, 'eval_steps_per_second': 8.767, 'epoch': 5.0}